# Lab3 Assignment

In [54]:
#Import Libraies
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [55]:
#Read file to Dataframe
Data = pd.read_excel("Cleaned_Update_2017.xlsx",sheetname=0)

### To get yahoo finance data

In [56]:
import requests
import json
import urllib

def get_symbol(name):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(name)

    result = requests.get(url).json()
    
    for x in result['ResultSet']['Result']:
        return x['symbol']
        
ticker=[]
cmp_name = Data.Entity
  
for  xyz in cmp_name:
    ticker.append(get_symbol(xyz))
        
print(ticker)

['AOL-U.TI', 'ADP', None, 'CIT-U.TI', None, 'HPQ', None, 'KDDIY', None, 'AOL-U.TI', None, None, None, 'GPS', None, None, None, None, None, 'AMTD', 'FIS', 'DNPCF', None, None, None, None, None, 'AT', 'SBUX', None, None, None, '^UMIAMIFL', None, None, None, None, None, None, None, None, None, None, None, None, 'LYNS', None, None, 'IGRW', None, None, None, None, 'HTLD', None, 'AT', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'PPB.L', 'JPM-PB', None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'CIT-U.TI', None, None, 'GB00B1Z54R10.L', None, None, None, 'MBSS.JK', None, None, None, None, None, None, None, 'NHS', None, None, None, None, 'SAND-P.V', None, None, None, None, None, None, None, None, None, 'GPN', None, None, None, None, None, 'KTC.DU', None, None, 'ATVI', None, None, None, None, 'KCT.V', 'K55306BA2822.KS', None, None, None, None, None, None, 'CIT-U.TI', None, None, '2GF.F', 'NTDOY', 'TWTR', 'AAPL', 'O

### With the ticker symbol, getting the stock price for the specified time frame

In [58]:
start = dt.datetime(2004,1,1)
end = dt.datetime(2017,6,6)
data_source = 'yahoo'
finance_data = pdr.DataReader(ticker, data_source, start, end)

/Users/nandinirajeswaran/Applications/anaconda/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: 'AOL-U.TI', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/nandinirajeswaran/Applications/anaconda/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: None, replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/nandinirajeswaran/Applications/anaconda/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: 'CIT-U.TI', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/nandinirajeswaran/Applications/anaconda/lib/python3.6/site-packages/pandas_datareader/yahoo/daily.py:136: SymbolWarning: Failed to read symbol: '^UMIAMIFL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/nandinirajeswaran/Applications/anaconda/lib/python3.6/site-packages/pandas

In [59]:
finance_data

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 3567 (major_axis) x 63 (minor_axis)
Items axis: Adj Close to Volume
Major_axis axis: 2004-01-01 00:00:00 to 2017-06-07 00:00:00
Minor_axis axis: ADP to 1880.HK

### To get the finance data into a dataframe

In [65]:
# The index in this DataFrame is the major index of the finance_data.
final_data = finance_data.ix['Close']

# Getting all weekdays as no stocks on weekends
all_weekdays = pd.date_range(start=start, end=end, freq='B')

# To align the existing prices in adj_close with all_weekdays as the new index using reindex
final_data = final_data.reindex(all_weekdays)


In [71]:
final_data.head()

,2004-01-31 00:00:00,2005-01-31 00:00:00,2006-01-31 00:00:00,2007-01-31 00:00:00,2008-01-31 00:00:00,2009-01-31 00:00:00,2010-01-31 00:00:00,2011-01-31 00:00:00,2012-01-31 00:00:00,2013-01-31 00:00:00,2014-01-31 00:00:00,2015-01-31 00:00:00,2016-01-31 00:00:00,2017-01-31 00:00:00,2018-01-31 00:00:00
ADP,32.813820,34.064935,35.050695,36.863325,40.408635,35.680073,33.713959,37.884942,45.168799,49.372064,62.635004,73.676731,84.028088,90.823439,101.266322
HPQ,11.182107,9.282525,11.431419,16.083477,21.027115,19.370576,18.872159,20.788701,15.047071,8.861357,10.792335,15.770325,13.679339,13.505929,17.734483
KDDIY,NaN,NaN,NaN,NaN,NaN,5.760977,4.477570,4.312767,5.716693,5.705365,8.373545,9.948712,11.916003,14.199206,13.273333
GPS,20.274500,21.430397,19.620000,18.361594,18.685913,16.659802,17.703267,20.624862,19.291825,30.194582,39.342302,40.946255,34.085259,24.304087,24.133908
AMTD,15.622000,12.953095,17.794405,17.797131,18.087817,16.548333,17.460120,17.583518,18.069206,17.205378,25.242143,32.671235,35.333307,33.768696,39.345977


### To get the stock price(average) for the year

In [68]:
final_data = final_data.groupby(pd.TimeGrouper(freq='12M')).mean()

In [70]:
#Transpose of the dataframe
final_data = final_data.T

### To get the data into a dataframe with columns - Symbols, Price, Date

In [72]:
Data_Final = pd.DataFrame(columns=['Symbol','Date','Price'])
d =0
for i in range(len(close)) : 
    ts = close.index[i]
    for j in range(len(close.iloc[i])): 
        time_frame = pd.to_datetime(str(close.columns.values[j]))
        date = time_frame.strftime('%Y-%m-%d')
        Data_Final.loc[d]=( ts,  date , close.iloc[i][j])
        d=d+1

In [73]:
#Removing rows with null values
Data_Final = Data_Final.dropna(axis=0,how='any')

In [74]:
Data_Final.head()

,Symbol,Date,Price
0,ADP,2004-01-31,32.813820
1,ADP,2005-01-31,34.064935
2,ADP,2006-01-31,35.050695
3,ADP,2007-01-31,36.863325
4,ADP,2008-01-31,40.408635


### Get ticker array to a dataframe and merge it with the existing dataframe

In [75]:
data_ticker = pd.DataFrame(ticker) 
data_ticker['Ticker_Symbol'] = pd.Series(ticker, index=data_ticker.index)

In [76]:
Data['Ticker_symbol']= data_ticker['Ticker_Symbol']

In [77]:
#Drop empty columns(axis=1 for columns and axis=0 for rows; 
#how='all' to drop if all values are nan and how='any' to drop if any value is nan)
Data = Data.dropna(axis=0,how='any')

In [78]:
Data.head()

,Entity,YEAR,records lost,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY,Ticker_symbol
0,AOL,2004,92000000,web,inside job,Just email address/Online information,AOL-U.TI
1,Automatic Data Processing,2005,125000,financial,poor security,SSN/Personal details,ADP
3,Citigroup,2005,3900000,financial,lost / stolen device,Credit card information,CIT-U.TI
5,Hewlett Packard,2006,200000,"tech, retail",lost / stolen device,SSN/Personal details,HPQ
7,KDDI,2006,4000000,telecoms,hacked,Just email address/Online information,KDDIY


In [79]:
Data.to_csv('Lab3_Finance1.csv', index=False, encoding='utf-8')

In [81]:
Data_Final.to_csv('Lab_Finance_Data_Final.csv', index=False, encoding='utf-8')

### Version1:
https://public.tableau.com/profile/nandini6420#!/vizhome/Lab3_Financial_Impact_NR/Version1

This version gives the information that the financial markets punish security breaches. The chart can have more clarity at the entity level.

### Version2:
https://public.tableau.com/profile/nandini6420#!/vizhome/Lab3_Financial_Impact_NR_V2_0/Version2

This version speakes about the data breaches on the enity level across the records lost and close stock Price. It can have a better representation.

### Version3:
https://public.tableau.com/profile/nandini6420#!/vizhome/Lab3_Financial_Impact_NR_V3_0/Final_Version

This version clearly depicts the scenario across the year and entity. Thus, financial markets punish security breaches